# Justin Butler
## AAE 537: HW 4

This assignment was created in Jupyter Notebook and formatted via $\LaTeX$

In [1]:
%%javascript
// Making sure the outputs display correctly
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

In [2]:
# Importing required packages for this homework
import numpy as np
import matplotlib.pyplot as plt
#from IPython.display import HTML, display
#import tabulate
%matplotlib inline

### i)
For the first part of this assignment, I will be labeling out how to find the stagnation temperature at each station in the engine as well as the fuel/air ratios in the burner and afterburner. We will assume that q, $\beta$, CPR, $C_p$, $\Delta H_B$ (of the fuel), $\dot{m}_0$, T$_t4$, and T$_t7$ are all known terms. We will also assume we know the efficiencies of the engine, that is, we know $\eta_c, \eta_b, \eta_t, \eta_{ab},$ and $\eta_n$.

#### Inlet
For the inlet, we will use the use the atmospheric conditions at a given altitude as given by the atmospheric model NASA provides at https://www.grc.nasa.gov/WWW/k-12/airplane/atmosmet.html. 

From static pressure we can also find the mach. Assuming $\gamma=1.33$ (or that $\gamma$ is at least a known value) and assuming the ambient air acts like an ideal gas we know:
$$M_0 = \sqrt{\frac{2\cdot q}{\gamma\cdot P_{s0}}}$$

Now with static temperature and mach, we find total temperature at the inlet as:
$$T_{t0} = T_{s0} \left(1 + \frac{\gamma-1}{2}M_0^2\right)$$
From this, we will assume an adiabatic intake - thus $T_{t0} = T_{t2}$. Next, by using the inlet recovery per Mil Std 5008B we can define the piecewise function for pressure ratio as:
$\frac{P_{t2}}{P_{t0}} = \begin{cases}
  1 & \text{for }M_0 <  1\\
  1 - 0.075(M_0-1)^{1.35} & \text{for } 1< M_0 < 5\\
  \frac{800}{M_0^4+935} & \text{for } M_0 > 5 
\end{cases}$

Since stagnation pressure can be calculated as the sum of static and dynamic pressures, we find $P_{t0} = P_{s0} + q$ where $P_{s0}$ is the pressure found from the atmospheric model. From this we can find the total pressure at station 2 from $P_{t2} = P_{t0} \cdot \left(P_{t2}/P_{t0}\right)$.
#### Compressor

For the compressor we can find the stagnation temperature and stagnation pressure for the air by:
$$T_{t3} = T_{t2}\left[1+\frac{1}{\eta_c}\cdot\left(CPR^{(\gamma-1)/\gamma}-1\right)\right]$$
$$P_{t3} = P_{t2}\cdot CPR$$

We will also want to find the massflow running through the core. Since we know $\beta$ and the inlet massflow $\dot{m}$, the massflow going through the core would be:
$$ \dot{m}_c = \frac{\dot{m}}{\beta+1}$$
#### Burner

Moving onto the burner, $T_{t4}$ is a given value and we will assume that $P_{t4} = P_{t3}$. We can find the fuel to air ratio by:
$$f = \frac{T_{t4} - T_{t3}}{\Delta H_B}\cdot C_p$$
#### Turbine

Station 5, within the figure, looks like it occurs before the re-mixing of the core and bypass streams. Thus, we will assume that the massflow out of the core at stage 5 is $\dot{m}_c + \dot{m}_f$. With this value, we will use powerbalance between the compressor and the turbine to determine the total temperature at the end of the turbine. Indeed:
$$T_{t5} = T_{t4} - \frac{\dot{m}_c\cdot C_p\cdot(T_{t3}-T_{t2})}{(\dot{m}_c+\dot{m}_f)\cdot C_p}$$
And from this, we can use isentropic relations to find:
$$P_{t5} = P_{t4}\left(\frac{T_{t5}}{T_{t4}}\right)^\frac{\gamma}{(\gamma-1)}$$
#### Mixer

Looking at the figure, by the time we reach station 6 the bypass flow has rejoined. Thus, the massflow now includes the bypass flow which we will need to account for. For this, we will perform weighted averages.
$$T_{t6} = \frac{\dot{m}_b}{\dot{m}_c+\dot{m}_f+\dot{m}_b}\cdot T_{t2} + \frac{\dot{m}_c + \dot{m}_f} {\dot{m}_c+\dot{m}_f+\dot{m}_b} \cdot T_{t5}$$
$$P_{t6} = \frac{\dot{m}_b}{\dot{m}_c+\dot{m}_f+\dot{m}_b}\cdot P_{t2} + \frac{\dot{m}_c + \dot{m}_f} {\dot{m}_c+\dot{m}_f+\dot{m}_b} \cdot P_{t5}$$
#### Afterburner

At the afterburner, $T_{t7}$ is a given value and we will again assume that $P_{t7}=P_{t6}$. Thus, the fuel to air ratio is:
$$f_{ab} = \frac{1-T_{t5}}{(\Delta H_B/C_p) - T_{t7}}$$

#### Nozzle

A perfectly expanded nozzle means that $P_9 = P_0$ and that $P_{t9} = P_{t7}$. We will also assume this is an adiabatic process and thus $T_{t9} = T_{t7}$. We can then find the velocity at the exit by:
$$V_9 = \sqrt{2C_pT_{t7}\left(1-\left(\frac{P_9}{P_{t7}}\right)^\frac{\gamma-1}{\gamma}\right)}$$


### ii)
The following is the routine that solves for the engine conditions and specific thrust and fuel consumption at a given flight condition and known temperature limits.

In [22]:
class engine:
    def help():
        print("Engine Class:")
        print("The engine class takes as input several parameters pertaining to a \n"
             "gas turbine engine and calculates information from those starting parameters.\n"
             "The required inputs are in the following order:\n"
              "1. Altitude\n"
              "2. dynamic pressure\n"
              "3. Flow Bypass Ratio \n"
              "4. Compressor Pressure Ratio \n"
              "5. ratio of specific heats\n"
              "6. Total temperature at end of combustor\n"
              "7. Total temperature at exit of the afterburner\n"
              "8. A 5 element list containing the efficiences (c,b,t,ab,n)\n"
              "9. Heating value of the fuel"
             )
        print("\nThese values will be input as values after the variable is defined.\n"
        "The program will then run and generate values. These include arrays of temperatures,\n"
        "pressures, mach, areas, and values for the mass flows throughout. There will also be\n"
        "values calculated for the specific thrust and fuel consumption.\n")
        print("For errors in this code, you can contact Justin on Github as user BananaBonanza")
    def __init__(self):
        inputTerms = ["altitude", "dynamic pressure", "Flow Bypass Ratio", 
                      "compressor Pressure Ratio","ratio of specific heats",
                      "total temperature at end of combustor",
                      "total temperature at exit of the afterburner",
                      "the differet efficiences",
                      "Heating value of the fuel"]
        inputVal = []
        for item in inputTerms:
            print("Input the value of " + item)
            if item != "the differet efficiences":
                inputVal.append(float(input()))
            else:
                effTerms = ["Compressor: ", "Burner: ", "Turbine: ", "Afterburner: ", "Nozzle: "]
                effVal = []
                for jtem in effTerms:
                    print("  " + jtem)
                    eff = input()
                    print(eff)
                    effVal.append(eff)
                inputVal.append(effVal)
        print(inputVal)
        
        
test = engine()
    
    

Input the value of altitude
1
Input the value of dynamic pressure
1
Input the value of Flow Bypass Ratio
1
Input the value of compressor Pressure Ratio
1
Input the value of ratio of specific heats
1
Input the value of total temperature at end of combustor
1
Input the value of total temperature at exit of the afterburner
1
Input the value of the differet efficiences
  Compressor: 
1
1
  Burner: 
1
1
  Turbine: 
1
1
  Afterburner: 
11
11
  Nozzle: 
12
12
Input the value of Heating value of the fuel
1
[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ['Compressor: ', 'Burner: ', 'Turbine: ', 'Afterburner: ', 'Nozzle: '], 1.0]


A = (mdot R static_T)/(static P and V)